In [1]:
import itertools
import re
import json
import random

In [2]:
class CNF:
    def __init__(self, dimacs):
        dimacs_tokens = re.split('\s+', dimacs)
        self.num_of_vars = int(dimacs_tokens[2])
        self.num_of_clauses = int(dimacs_tokens[3])
        clauses_dimacs = [int(x) for x in dimacs_tokens[4:]]
        self.clauses = [list(clause) for is_zero, clause 
                    in itertools.groupby(clauses_dimacs, lambda x: x == 0) 
                    if not is_zero]
        
    def evaluate(self, valuation):
        for clause in self.clauses:
            clause_sat = False
            for literal in clause:
                val_idx = abs(literal) - 1
                if literal > 0 and valuation[val_idx] or literal < 0 and not valuation[val_idx]:
                    clause_sat = True
                    break
            if not clause_sat:
                return False
        return True

In [3]:
class Problem:
    def __init__(self, file_name):
        try:
            with open(file_name, 'r') as f:
                dimacs_list = json.load(f)
                self.cnf_list = [CNF(dimacs) for dimacs in dimacs_list]
                self.num_of_vars = self.cnf_list[0].num_of_vars
        except IOError:
            print(f'Error opening file {file_name}')
            exit(1)

In [8]:
class BFSolver:
    def solve(self, problem):
        all_valuations = itertools.product([True, False], repeat=problem.num_of_vars)
        min_num_of_sat = float('inf')
        best_valuation = None
        iters = 0
        for valuation in all_valuations:
            iters += 1
            num_of_sat = 0
            for formula in problem.cnf_list:
                if formula.evaluate(valuation):
                    num_of_sat += 1
            if num_of_sat < min_num_of_sat:
                min_num_of_sat = num_of_sat
                best_valuation = valuation
        return min_num_of_sat, best_valuation

In [9]:
p = Problem('problem_instances/1.json')
bfs_solver = BFSolver()
print(bfs_solver.solve(p))

(30, (True, True, True, False, False, False, False, False, True, True))
